In [1]:
import findspark
findspark.init()
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import datetime
import time
import numpy as np
import pandas as pd
import talib
from alice_blue import *
from alice_blue import AliceBlue
from config import Credentials
kafka_topic_name = "OHLC_DATA"
kafka_bootstrap_servers = 'localhost:9092'

In [2]:
UserName = Credentials.UserName.value
PassWord = Credentials.PassWord.value
SecretKey = Credentials.SecretKey.value
AppId = Credentials.AppId.value
f2 = 1997

access_token = AliceBlue.login_and_get_access_token(
    username=UserName, password=PassWord, twoFA=f2, api_secret=SecretKey, app_id=AppId
)


SCRIPT_LIST = ["Nifty Bank"]

In [3]:
alice = AliceBlue(
    username=UserName,
    password=PassWord,
    access_token=access_token,
    master_contracts_to_download=["NSE"],
)


In [4]:
spark = SparkSession \
    .builder \
    .appName("Structured Streaming ") \
    .master("local[*]") \
    .getOrCreate()


In [5]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic_name) \
    .option("startingOffsets", "latest") \
    .load().selectExpr("CAST(value AS STRING)", "timestamp")


In [6]:
stock_data_schema = "timestamp STRING, open float, high Float, low Float, close Float"
df2 = df \
    .select(from_csv(col("value"), stock_data_schema)
            .alias("stock"))
df2 = df2.select("stock.*")
df2 = (df2.withColumn('timestamp',regexp_replace('timestamp',"'",""))
          .withColumn('timestamp',to_timestamp(col('timestamp'))))


df2.createOrReplaceTempView('ohlc_data')
all_data = spark.sql("SELECT * FROM ohlc_data")
stock_data_write_stream = all_data.writeStream \
    .trigger(processingTime='1 seconds') \
    .outputMode("append") \
    .option("truncate", "false") \
    .format("memory") \
    .queryName("ohlcData") \
    .start()


In [24]:
spark.sql('select * from ohlcData').show(200)

+-------------------+--------+--------+--------+--------+
|          timestamp|    open|    high|     low|   close|
+-------------------+--------+--------+--------+--------+
|2022-02-15 12:35:00|37208.95|37261.75|37202.65|37259.65|
|2022-02-15 12:36:00| 37258.3|37267.85| 37254.2|37263.35|
|2022-02-15 12:37:00| 37266.5| 37289.5|37264.15|37279.05|
|2022-02-15 12:38:00|37277.85| 37280.1| 37239.5|37265.05|
|2022-02-15 12:39:00|37270.25|37276.55| 37249.7|37266.45|
|2022-02-15 12:40:00| 37262.8| 37269.0| 37250.1| 37262.3|
|2022-02-15 12:41:00| 37263.6| 37269.1|37246.95| 37248.7|
|2022-02-15 12:42:00|37248.15| 37257.6|37230.75| 37248.0|
|2022-02-15 12:43:00|37254.05| 37254.1|37236.15|37236.15|
+-------------------+--------+--------+--------+--------+



In [6]:
def rsi(df):
    n=14
    "function to calculate RSI"
    delta = df["close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

In [7]:
def rsi_tradingview(ohlc, period: int = 14, round_rsi: bool = True):
    

    delta = ohlc.diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return pd.DataFrame(np.round(rsi, 2)) if round_rsi else rsi

In [13]:
initial_time = spark.sql('select * from ohlcData').take(1)[0]['timestamp']

In [63]:
 initial_time=initial_time+datetime.timedelta(hours=2)

In [23]:
initial_time

datetime.datetime(2022, 2, 15, 12, 35)

In [26]:
spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15").show()

+-------------------+--------+--------+--------+--------+
|          timestamp|    open|    high|     low|   close|
+-------------------+--------+--------+--------+--------+
|2022-02-15 12:35:00|37208.95|37261.75|37202.65|37259.65|
|2022-02-15 12:36:00| 37258.3|37267.85| 37254.2|37263.35|
|2022-02-15 12:37:00| 37266.5| 37289.5|37264.15|37279.05|
|2022-02-15 12:38:00|37277.85| 37280.1| 37239.5|37265.05|
|2022-02-15 12:39:00|37270.25|37276.55| 37249.7|37266.45|
|2022-02-15 12:40:00| 37262.8| 37269.0| 37250.1| 37262.3|
|2022-02-15 12:41:00| 37263.6| 37269.1|37246.95| 37248.7|
|2022-02-15 12:42:00|37248.15| 37257.6|37230.75| 37248.0|
|2022-02-15 12:43:00|37254.05| 37254.1|37236.15|37236.15|
|2022-02-15 12:44:00| 37238.9| 37258.4| 37238.4|37253.05|
|2022-02-15 12:45:00| 37257.0|37268.55|37249.35|37249.35|
|2022-02-15 12:46:00| 37249.3| 37263.6|37231.05|37231.05|
|2022-02-15 12:47:00| 37234.2| 37279.8| 37234.2| 37279.8|
|2022-02-15 12:48:00| 37283.7|37302.45|37278.55|37302.45|
|2022-02-15 12

## RSi

In [20]:
while True:
    ohlc_data = spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15")
    ohlc_data_pd = ohlc_data.toPandas().set_index('timestamp')
    if(len(ohlc_data_pd) < 15):
        #print('if continue')
        continue
    else:
        print('else run')
        print(rsi(ohlc_data_pd.iloc[:15]).tail(1).to_string())
        initial_time+= datetime.timedelta(minutes=1)


KeyboardInterrupt: 

## rsi tradingview

In [ ]:
while True:
    ohlc_data = spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15")
    ohlc_data_pd = ohlc_data.toPandas().set_index('timestamp')
    if(len(ohlc_data_pd) < 15):
        continue
    else:
        print(rsi_tradingview(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string())
        initial_time+= datetime.timedelta(minutes=1)


## Talib

In [ ]:
while True:
    ohlc_data = spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15")
    ohlc_data_pd = ohlc_data.toPandas().set_index('timestamp')
    if(len(ohlc_data_pd) < 15):
        continue
    else:
        print(talib.RSI(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string())
        initial_time+= datetime.timedelta(minutes=1)


In [28]:
rsi2=None
while True:
    ohlc_data = spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15")
    ohlc_data_pd = ohlc_data.toPandas().set_index('timestamp')
    if(len(ohlc_data_pd) < 15):
        continue
    else:
        
        rsi1=talib.RSI(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string()
        if rsi2 and rsi1>rsi2 and rsi1>40:
            break40=True
            
        if rsi2 and rsi1<rsi2 and rsi1<60:
            break60 = True
            
        rsi2=talib.RSI(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string()
        
        initial_time+= datetime.timedelta(minutes=1)
        print(break60, break40)



KeyboardInterrupt: 

In [ ]:
isAbove60 = None
isBelow40 = None

break60 = None
break40 = None

while True:
    ohlc_data = spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15")
    ohlc_data_pd = ohlc_data.toPandas().set_index('timestamp')
    if(len(ohlc_data_pd) < 15):
        continue4
    else:
        rsi=talib.RSI(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string()
        if rsi > 60:
           isAbove60bove60Above60 = True
        else:
isAbove60        isAbove60 = False
        if rsi < 40:
            isBelow40 = True
        else:
            isBelow40 = False
        
        if 
        initial_time+= datetime.timedelta(minutes=1)


In [ ]:
break60 = False
break40 = False

if break60 == Ture:
    placeorder
    break60=False
    
if break40 == True:
    placeorder
    break40 = False

In [ ]:
while True:
    ohlc_data = spark.sql(f"select * from ohlcData where timestamp >='{initial_time}' limit 15")
    ohlc_data_pd = ohlc_data.toPandas().set_index('timestamp')
    if(len(ohlc_data_pd) < 15):
        continue
    else:
        rsi = talib.RSI(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string() 
        print(talib.RSI(ohlc_data_pd["close"].iloc[:15]).tail(1).to_string())
        initial_time+= datetime.timedelta(minutes=1)


In [5]:
alice.place_order(transaction_type = TransactionType.Buy,
                     instrument = alice.get_instrument_by_symbol('NSE', 'INFY'),
                     quantity = 1,
                     order_type = OrderType.Market,
                     product_type = ProductType.Intraday,
                     price = 0.0,
                     trigger_price = None,
                     stop_loss = None,
                     square_off = None,
                     trailing_sl = None,
                     is_amo = False)

{'status': 'success',
 'message': 'Order placed successfully',
 'data': {'oms_order_id': '220214000179120'}}